# Using TensorFlow with Amazon SageMaker's training and hosting services

1. [Introduction](#Introduction)
2. [Setup](#Setup)
3. [Train the model](#Train-the-model)
4. [Host the model](#Host-the-model)
5. [Test the model](#Test-the-model)
6. [Clean up](#Clean-up)

## Introduction

The previous lab performed training and prediction directly in the Jupyter notebook environment. With this lab, we transition to leveraging SageMaker's managed training and hosting services. To accomplish this, we use [Amazon SageMaker's TensorFlow container](https://sagemaker.readthedocs.io/en/stable/using_tf.html), which lets you provide your training code as a Python script. The container also provides a flexible way for you to customize how inference inputs and outputs are handled over a REST interface.

## Setup

Before preparing the data, there are some initial steps required for setup. To train the image classification algorithm on Amazon SageMaker, we need to setup and authenticate the use of AWS services. To begin with, we need an AWS account role with SageMaker access. Here we will use the execution role the current notebook instance was given when it was created.  This role has necessary permissions, including access to your data in S3.

In [ ]:
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print(role)
sess = sagemaker.Session()

We also need to identify the S3 bucket that you want to use for providing training and validation datasets.  It will  be used to store the tranied model artifacts as well. In this notebook, we use a default bucket for use with SageMaker in your account. Alternatively, you could use whatever bucket you would like.  We use an object prefix to help organize the bucket content.

In [ ]:
bucket = sess.default_bucket() # or use your own custom bucket name
prefix = 'DEMO-TF-image-classification-birds'

# Data Preparation

This notebook assumes you have already downloaded and unpacked the dataset into your notebook instance as part of the first lab.

## Set some parameters for the rest of the notebook to use

Here we define a few parameters that help drive the rest of the notebook.  For example, `SAMPLE_ONLY` is defaulted to `True`. This will force the notebook to train on only a handful of species.  Setting `SAMPLE_ONLY` to false will make the notebook work with the entire dataset of 200 bird species.  This makes the training a more difficult challenge, and you will need to tune parameters and run more epochs.

An `EXCLUDE_IMAGE_LIST` is defined as a mechanism to address any corrupt images from the dataset and ensure they do not disrupt the process.

In [ ]:
import pandas as pd
import boto3
import json
import pandas as pd

runtime = boto3.client(service_name='runtime.sagemaker')

import matplotlib.pyplot as plt
%matplotlib inline

# To speed up training and experimenting, you can use a small handful of species.
# To see the full list of the classes available, look at the content of CLASSES_FILE.
SAMPLE_ONLY  = True
CLASSES = [13, 17] #, 35, 36, 47, 68, 73, 75, 87, 95, 120, 179, 192]

# Otherwise, you can use the full set of species
if not SAMPLE_ONLY:
    CLASSES = []
    for c in range(200):
        CLASSES += [c + 1]
    prefix = 'DEMO-TF-image-classification-birds-full'

BASE_DIR   = 'CUB_200_2011/'
IMAGES_DIR = BASE_DIR + 'images/'
CLASSES_FILE = BASE_DIR + 'classes.txt'

EXCLUDE_IMAGE_LIST = ['087.Mallard/Mallard_0130_76836.jpg']

SPLIT_RATIOS = (0.6, 0.2, 0.2)

CLASS_COLS      = ['class_number','class_id']

JOB_PREFIX     = 'mpr-tf-ic'

Show the list of bird species or dataset classes the model will be trained to predict.

In [ ]:
classes_df = pd.read_csv(CLASSES_FILE, sep=' ', names=CLASS_COLS, header=None)
criteria = classes_df['class_number'].isin(CLASSES)
classes_df = classes_df[criteria]

class_name_list = sorted(classes_df['class_id'].unique().tolist())
print(class_name_list)

# Create train/val/test dataframes from our dataset

In this lab, we've moved the train/test split logic to a utilities script to keep the focus of the notebook on the actual training and hosting steps.

In [ ]:
from utils import split
train_df, val_df, test_df = split.get_train_val_dataframes(BASE_DIR, CLASSES, EXCLUDE_IMAGE_LIST, SPLIT_RATIOS)

# Prepare the data channels for Amazon SageMaker
When using Amazon SageMaker's managed training service, you need to provide the datasets to the training algorithm. This is primarily handled via populating S3 buckets, and by indicating the location of data channels such as train, test, and validation. You also need to consider the data format. In our case, to keep things simple, we will populate the data channels with folders containing the original JPG images organized by class folders.

In [ ]:
CHANNEL_FOLDER = 'data'

### Populate local data channels
Here we populate the local channel folders, and we handle re-populating in case we have already run through this notebook with a different subset of bird species. The local content will be synchronized to our s3 bucket for access by the SageMaker training service. Note that if you are running this notebook with a large number of bird species, this step could take several minutes. You can skip these cells if you are re-running this notebook after already having populated these folders. 

In [ ]:
import os
cwd = os.getcwd()

In [ ]:
def copy_files_for_channel(df, channel_name, verbose=False):
    print('\nCopying files for {} images in channel: {}...'.format(df.shape[0], channel_name))
    for i in range(df.shape[0]):
        target_fname = df.iloc[i]['image_file_name']
        if verbose:
            print(target_fname)
        !cp $cwd/$IMAGES_DIR/$target_fname $cwd/$CHANNEL_FOLDER/$channel_name/$target_fname

In [ ]:
if os.path.exists(CHANNEL_FOLDER):
    print('About to remove {}/{}'.format(cwd, CHANNEL_FOLDER))
    perform_delete = input('Are you sure you want to proceed (yes/no)?')
    if perform_delete == 'yes':
        for ch in ['train', 'test', 'validation']:
            !rm -i -rf $CHANNEL_FOLDER/$ch/*
            !rm -i -rf $CHANNEL_FOLDER/$ch
            !mkdir  $CHANNEL_FOLDER/$ch

        for c in class_name_list:
            !mkdir $CHANNEL_FOLDER/train/$c
            !mkdir $CHANNEL_FOLDER/validation/$c
            !mkdir $CHANNEL_FOLDER/test/$c
            
        copy_files_for_channel(val_df,   'validation')
        copy_files_for_channel(test_df,  'test')
        copy_files_for_channel(train_df, 'train')
else:
    print('Channel folder does not yet exist. Creating it and subfolders for each channel.')
    os.mkdir(CHANNEL_FOLDER)
    os.mkdir(CHANNEL_FOLDER + '/validation')
    os.mkdir(CHANNEL_FOLDER + '/test')
    os.mkdir(CHANNEL_FOLDER + '/train')
    for c in class_name_list:
        os.mkdir('{}/{}/{}'.format(CHANNEL_FOLDER, 'validation', c))
        os.mkdir('{}/{}/{}'.format(CHANNEL_FOLDER, 'test', c))
        os.mkdir('{}/{}/{}'.format(CHANNEL_FOLDER, 'train', c))
    
    copy_files_for_channel(val_df,   'validation')
    copy_files_for_channel(test_df,  'test')
    copy_files_for_channel(train_df, 'train')

### Upload content of each data channel to S3
Note that for local mode training, this copy to s3 is not necessary. SageMaker local mode is helpful for early iterations of the development of a new model. Once the approach is more stable, you typically then use SageMaker training jobs on larger sets of data and with additional epochs.

In [ ]:
if input('Are you sure you want to replace your s3 images?') == 'yes':
    print('Clearing out s3://{}/{}/'.format(bucket, prefix))
    for ch in ['train', 'test', 'validation']:
        !aws s3 rm --quiet --recursive s3://$bucket/$prefix/$ch/
    print('\nSynchronizing local data channels with s3...')
    !aws s3 sync $CHANNEL_FOLDER s3://$bucket/$prefix/

print('contents of s3://{}/{}/'.format(bucket, prefix))
!aws s3 ls s3://$bucket/$prefix/

# Train the model 
When using SageMaker's TensorFlow container, the custom TensorFlow training code is provided via a Python script in a separate file that gets passed to SageMaker. For our example, that script is shown below for completeness in the notebook. Study that code before proceeding to the actual training. Pay attention to any differences from the code you used in the first lab when training directly in the notebook:

- Use of script parameters to make the code more flexible. We will take advantage of these parameters later on when doing automatic model tuning.
- Copying of `inference.py` and `requirements.txt` to the code directory for use with TensorFlow Serving at inference time.
- Different approach to model saving to be compatible with SageMaker's use of TensorFlow Serving.

In [1]:
!pygmentize 'code/train-mobilenet.py'

# Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License"). You
# may not use this file except in compliance with the License. A copy of
# the License is located at
#
#     http://aws.amazon.com/apache2.0/
#
# or in the "license" file accompanying this file. This file is
# distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF
# ANY KIND, either express or implied. See the License for the specific
# language governing permissions and limitations under the License.

import tensorflow as tf
import tensorflow.keras

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
LAST_FROZEN_LAYER = 20

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

In [2]:
!pygmentize code/inference.py

# Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License"). You
# may not use this file except in compliance with the License. A copy of
# the License is located at
#
#     http://aws.amazon.com/apache2.0/
#
# or in the "license" file accompanying this file. This file is
# distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF
# ANY KIND, either express or implied. See the License for the specific
# language governing permissions and limitations under the License.

import io
import json
import numpy as np
from collections import namedtuple
from PIL import Image

import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from numpy import argmax

import logging
logging.basicConfig(level=logging.DEBUG)
logging.getLogger().setLevel(logging.DEBUG)
tf.logging.set_verbosity(tf.logging.ERROR)

HEIGHT 

In [3]:
!cat ./code/requirements.txt

Pillow
numpy

When using local mode, this simple setup script is required.

In [ ]:
!/bin/bash ./setup.sh

## Create the SageMaker training job using the TensorFlow container
Here we establish the Tensorflow estimator object. Note that this notebook code is designed to support SageMaker remote training jobs as well as local mode training. Set `local = False` to use SageMaker's training service on separate managed training instances. Metric definitions are provided so that you can visualize metrics from the SageMaker console as well as from CloudWatch. These same metrics can be used when optimizing your model with automatic model tuning.

For this lab, run the whole notebook with `local = True` first. Once you have understood all that is going on, and have seen it work successfully, change to `local = False` and re-run the rest of the notebook.

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow
import logging

TF_FRAMEWORK_VERSION = '1.12'

local = False
if (local):
    train_instance_type = 'local'
    serve_instance_type = 'local'
else:
    train_instance_type = 'ml.p3.8xlarge' 
    serve_instance_type = 'ml.c5.xlarge'

hyperparameters = {'fine_tuning_epochs': 25, 
                   'dropout': 0.5,
                   'data_dir': '/opt/ml/input/data'}

metric_definitions=[{'Name' : 'validation:acc', 
                     'Regex': '.*step.* - val_acc: (.*$)'},
                    {'Name' : 'validation:loss', 
                     'Regex': '- val_loss: (.*?) '},
                   {'Name' : 'acc', 
                     'Regex': '.*step.* - acc: (.*?) '},
                    {'Name' : 'loss', 
                     'Regex': '.*step.* - loss: (.*?) '}]

estimator = TensorFlow(entry_point='train-mobilenet.py',
                       source_dir='code',
                       train_instance_type=train_instance_type,
                       train_instance_count=1,
                       hyperparameters=hyperparameters,
                       metric_definitions=metric_definitions,
                       role=sagemaker.get_execution_role(),
                       framework_version=TF_FRAMEWORK_VERSION, 
                       py_version='py3',
                       base_job_name=JOB_PREFIX,
                       script_mode=True)

Here we establish pointers to where each data channel is located. For local mode training, these will be in the local file system. For remote training, the s3 data channel copies are provided.

In [ ]:
data_dir = os.path.join(os.getcwd(), 'data')
print('Local data dir: {}'.format(data_dir))

if (local):
    train_in = f'file://{data_dir}/train'
    test_in  = f'file://{data_dir}/test'
    val_in   = f'file://{data_dir}/validation'
else:
    s3_base = 's3://{}/{}'.format(bucket, prefix)
    train_in = f'{s3_base}/train'
    test_in  = f'{s3_base}/test'
    val_in   = f'{s3_base}/validation'

inputs = {'train':train_in, 'test': test_in, 'validation': val_in}

print(inputs)

Here we tell the estimator to fit the model. For local mode, this training is performed in the current notebook instance. For remote training, new instances are launched and training is performed on those separate instances. No matter which technique you use, the training script can count on the same interface (script parameters, specific SageMaker environment variables, consistent location of data channels in `/opt/ml/input/data`, and saving to `/opt/ml/model`).

In [ ]:
estimator.fit(inputs) 

In [ ]:
print('Completed training job: {}'.format(estimator.latest_training_job.name))

# Host the model

In the simple case, once the fit method is complete, we can simply call deploy to publish the trained model. However, if you are iterating on various implementations of `inference.py`, you can avoid having to retrain the model by simply replacing the file inside of the model artifacts compressed zip file. This next cell runs a script that downloads the model artifacts, replaces the inference files, and uploads the new model artifacts zip for deployment.

In [ ]:
if not local:
    # re-deploy model with new inference.py without having to re-do training job
    from sagemaker.tensorflow.serving import Model

    training_job_name = estimator.latest_training_job.name
    model_artifacts = 's3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name)
    print(model_artifacts)

    !bash ./replace-inference.sh $model_artifacts

Here we deploy the updated model artifacts. If iterating on `inference.py` changes, this code re-creates the SageMaker model object and deploys it. Otherwise, simply deploy the model directly from the original estimator. You can use this approach once your `inference.py` code is stable.

In [ ]:
if not local:
    model = Model(model_data=model_artifacts, 
                  role=sagemaker.get_execution_role(),
                  framework_version=TF_FRAMEWORK_VERSION,
                  sagemaker_session=sess)

    predictor = model.deploy(initial_instance_count=1, 
                         instance_type=serve_instance_type)
else:
    predictor = estimator.deploy(initial_instance_count=1, 
                               instance_type=serve_instance_type,
                               endpoint_type='tensorflow-serving')

# Test the model 

In [ ]:
# If using an endpoint that was already in service, you can establish a predictor object with the following code.
#from sagemaker import RealTimePredictor
#predictor = RealTimePredictor(endpoint='<put-endpoint-name-here>')

In [ ]:
from tensorflow.keras.preprocessing import image
from numpy import argmax

## Predicting using the SageMaker TensorFlow endpoint
When calling your image classifier hosted in a SageMaker endpoint, the call to the predictor is slightly different than what you used in the first lab:

1. In lab 1, we preprocessed the image bytes, which is specific to our base MobileNet model. Instead, here we pass the image bytes directly. This hides the implementation detail of a model-specific preprocessing of the image as part of the endpoint. This gives us flexibility to switch to a different implementation down the road (perhaps ResNet50).
2. We indicate the content type for the rest call, in this case `application/x-image`.
3. The results come back as a json document with a `predictions` object.

The rest of the code remains the same.

In [ ]:
from IPython.display import Image, display
def predict_bird_from_file(fn, verbose=True):
    with open(fn, 'rb') as img:
        f = img.read()
    x = bytearray(f)
    
    predictor.content_type = 'application/x-image'
    predictor.serializer   = None
    
    results = predictor.predict(x)['predictions']

    predicted_class_idx = argmax(results)
    predicted_class = class_name_list[predicted_class_idx]
    confidence = results[0][predicted_class_idx]
    if verbose:
        display(Image(fn))
        print('Class: {}, confidence: {:.2f}'.format(predicted_class, confidence))
    del img, x
    return predicted_class_idx, confidence

In [ ]:
predict_bird_from_file('northern-cardinal-1.jpg')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.GnBu):
    plt.figure(figsize=(7,7))
    plt.grid(False)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), 
                                  range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.gca().set_xticklabels(class_name_list)
    plt.gca().set_yticklabels(class_name_list)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
from sklearn.metrics import confusion_matrix
def create_and_plot_confusion_matrix(actual, predicted):
    cnf_matrix = confusion_matrix(actual, np.asarray(predicted),labels=range(len(class_name_list)))
    plot_confusion_matrix(cnf_matrix, classes=range(len(class_name_list)))

### Assess prediction performance against validation and test datasets

In [ ]:
from IPython.display import Image, display

# Iterate through entire dataframe, tracking predictions and accuracy. For mistakes, show the image, and the predicted and actual classes to help understand
# where the model may need additional tuning.

def test_image_df(df):
    print('Testing {} images'.format(df.shape[0]))
    num_errors = 0
    preds = []
    acts  = []
    for i in range(df.shape[0]):
        fname = df.iloc[i]['image_file_name']
        act   = int(df.iloc[i]['class_id']) - 1
        acts.append(act)
        pred, conf = predict_bird_from_file(IMAGES_DIR + '/' + fname, verbose=False)
        preds.append(pred)
        if (pred != act):
            num_errors += 1
            print('ERROR on image index {} -- Pred: {} {:.2f}, Actual: {}'.format(i, 
                                                                   class_name_list[pred], conf, 
                                                                   class_name_list[act]))
            display(Image(filename=IMAGES_DIR + '/' + fname))
    return num_errors, preds, acts

In [ ]:
num_images = val_df.shape[0]
num_errors, preds, acts = test_image_df(val_df)
print('\nAccuracy: {:.2f}, {}/{}'.format(1 - (num_errors/num_images), num_images - num_errors, num_images))

In [ ]:
create_and_plot_confusion_matrix(acts, preds)

In [ ]:
num_images = test_df.shape[0]
num_errors, preds, acts = test_image_df(test_df)
print('\nAccuracy: {:.2f}, {}/{}'.format(1 - (num_errors/num_images), num_images - num_errors, num_images))

In [ ]:
create_and_plot_confusion_matrix(acts, preds)

### Test model against previously unseen images
Here we download images that the algorithm has not yet seen.

In [ ]:
predict_bird_from_file('northern-cardinal-2.jpg')
predict_bird_from_file('northern-cardinal-1.jpg')

# Clean up
Finally, and importantly, to avoid being billed for an idle endpoint, here we delete the SageMaker endpoint.

In [ ]:
predictor.delete_endpoint()